Created on Wed Jan 15 09:28:16 2020
<br>
Group 7
<br>
@author: G.H.

<h1>Group 7 - Images sociales<span class="tocSkip"></span>
        
<br>    
<center>Airbus Interiors - aircraft types model - Hackathon & SeatGuru<center>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Environment" data-toc-modified-id="Environment-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Environment</a></span><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Parameters" data-toc-modified-id="Parameters-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Parameters</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Functions</a></span></li></ul></li><li><span><a href="#Pre-processing" data-toc-modified-id="Pre-processing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Pre-processing</a></span><ul class="toc-item"><li><span><a href="#Create-directories" data-toc-modified-id="Create-directories-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Create directories</a></span></li><li><span><a href="#Train-test-split-and-read-images" data-toc-modified-id="Train-test-split-and-read-images-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Train-test split and read images</a></span></li></ul></li><li><span><a href="#Build,-save,-and-train-model" data-toc-modified-id="Build,-save,-and-train-model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Build, save, and train model</a></span></li></ul></div>

# Introduction
This notebook trains a model to predict Airbus aircraft types on images representing an aircraft interior. The images used for training come from 2 sources: Seatguru social media, and Hackathon data ("clean" images of Airbus aircraft interiors).

**Pre-processing**<span class="tocSkip"></span><br>
By reading and filtering the CSV file that contains the labels (Int, Ext, Ext-Int, Meal), we get the list of Interior labelled images. Then, the images are copied to directories (one per desired aircraft type), and split into train and test sets. If the data augmentation option is set to `True`, the train set will be enriched with new images (obtained by cropping / (de)zooming / rotating / flipping existing images).

**Model**<span class="tocSkip"></span><br>
We get weights from VGG16 pre-trained model, and add some layers (Conv2D, ReLU, MaxPooling2D, Flatten, and Dense) to predict the target classes (e.g.: 5 classes could be A320, A321, A330, A350, and A380).

**Out**<span class="tocSkip"></span><br>
After training, a folder is created in `Models` repository, containing the model in `h5` format, along with the corresponding labels stored in a `pickle` dictionary.

# Environment
To ensure a proper functioning of this code file, `python 3.6` or later version is required.

## Libraries

In [2]:
import warnings
warnings.filterwarnings('ignore')
from keras.applications.vgg16 import VGG16
from keras.callbacks import ReduceLROnPlateau
from keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.optimizers import SGD, Adam
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, InputLayer, ReLU, AveragePooling2D
from keras.models import Sequential, Model
from keras import backend as K
import keras
import pandas as pd
import random
import os
from PIL import Image
from shutil import copyfile

In [3]:
%load_ext watermark
%watermark -p keras,tensorflow,PIL

keras 2.3.1
tensorflow 1.13.1
PIL 6.2.0


## Parameters

In [32]:
project_path = './../'
seatguru_path = project_path + 'Interpromo2020/All Data/ANALYSE IMAGE/IMG SEATGURU/'
hackathon_path = project_path + 'InputsHackathon/'
stats_path = project_path + 'ImagesStats/'

model_name = 'Int_Airbus_Hackathon_Seatguru_F'

# Classes to predict
airbus_planes = ['A320', 'A321', 'A330', 'A350', 'A380']
nb_types = len(airbus_planes)

# Images parameters
size = (224, 224)
greys = False
apply_data_augmentation = False

## Functions

In [45]:
%run g7_functions_for_models.py

# Pre-processing
## Create directories

In [27]:
crea_path = project_path + 'G7_SEATGURU/Int/Airbus/'
new_paths = [crea_path + 'data_train', crea_path + 'data_test']
print(new_paths[0], new_paths[1])

# SEATGURU
# Read Seatguru annotated CSV from ImagesStas directory
df_seat_annot = pd.read_csv(stats_path + 'g7_SEATGURU_annotate.csv', sep=';')

# Create a directory for each class
create_dirs_seatguru_type(df_seat_annot, project_path, seatguru_path, crea_path,
                          aircraft_types=airbus_planes, view='Int', man='Airbus')

# NB: HACKATHON folders already exist

./../G7_SEATGURU/Int/Airbus/data_train ./../G7_SEATGURU/Int/Airbus/data_test
./../G7_SEATGURU/Int/Airbus/A320
A320: 166 images
./../G7_SEATGURU/Int/Airbus/A321
A321: 121 images
./../G7_SEATGURU/Int/Airbus/A330
A330: 257 images
./../G7_SEATGURU/Int/Airbus/A350
A350: 39 images
./../G7_SEATGURU/Int/Airbus/A380
A380: 46 images


## Train-test split and read images

In [36]:
# SEATGURU
# Split train and test
split_train_test_seatguru_type(
    new_paths=new_paths, path=crea_path, aircraft_types=airbus_planes)

# HACKATHON
# Split train and test
split_train_test_hack(new_paths, hackathon_path,
                      aircraft_types=airbus_planes, del_path=False)

In [ ]:
# Option: use data augmentation to enrich your train set
if apply_data_augmentation:
    %run g7_data_augmentation.py
    data_augmentation(train_path=new_paths[0], shape=size, save_format='jpeg', nb_win=2, coef_gen=2,
                      greys=False, rotation_range=20, shear_range=.2, zoom_range=.15, horizontal_flip=True)

In [37]:
# Image data generator
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = datagen.flow_from_directory(new_paths[0],
                                              target_size=size,
                                              color_mode='rgb',
                                              batch_size=32,
                                              class_mode='categorical',
                                              shuffle=True)

test_generator = datagen.flow_from_directory(new_paths[1],
                                             target_size=size,
                                             color_mode='rgb',
                                             batch_size=32,
                                             class_mode='categorical',
                                             shuffle=True)

Found 1240 images belonging to 5 classes.
Found 537 images belonging to 5 classes.


# Build, save, and train model

In [38]:
# Get the base pre-trained model
base_model = VGG16(weights='imagenet', include_top=False,
                   input_shape=(size[0], size[1], 1 if greys else 3))
x = base_model.output

# Add layers
x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Flatten()(x)  # vector

x = Dense(1024)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(512)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(256)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

# Last layer used to predict our classes
# Dense has the same number of neurons as the number of classes to predict
predictions = Dense(nb_types, activation='softmax')(x)

# Model to be trained
model = Model(inputs=base_model.input, outputs=predictions)

# Don't retrain pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model (should be done after setting layers to non-trainable)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [39]:
# Reduce learning rate when a metric has stopped improving
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',  # chosen metric
                              patience=2,  # number of epochs
                              verbose=1,
                              factor=0.5,
                              min_lr=0.00001)

In [40]:
# Train model
step_size_train = train_generator.n // train_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    epochs=10,
                    validation_data=test_generator,
                    callbacks=[reduce_lr])

Epoch 1/10
38/38 [==============================] - 292s 8s/step - loss: 1.5114 - accuracy: 0.4180 - val_loss: 8.3454 - val_accuracy: 0.2291
Epoch 2/10
38/38 [==============================] - 306s 8s/step - loss: 0.5212 - accuracy: 0.8320 - val_loss: 1.5423 - val_accuracy: 0.4264
Epoch 3/10
38/38 [==============================] - 315s 8s/step - loss: 0.1651 - accuracy: 0.9578 - val_loss: 1.7813 - val_accuracy: 0.4507
Epoch 4/10
38/38 [==============================] - 299s 8s/step - loss: 0.1154 - accuracy: 0.9627 - val_loss: 1.1391 - val_accuracy: 0.5717
Epoch 5/10
38/38 [==============================] - 304s 8s/step - loss: 0.0748 - accuracy: 0.9793 - val_loss: 1.1279 - val_accuracy: 0.5531
Epoch 6/10
38/38 [==============================] - 298s 8s/step - loss: 0.0808 - accuracy: 0.9770 - val_loss: 1.1660 - val_accuracy: 0.5493

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 7/10
38/38 [==============================] - 296s 8s/step - loss: 

In [46]:
# Save model and labels
os.makedirs(project_path + 'Models/' + model_name + '/', exist_ok=True)
save_model_classes(project_path + 'Models/',
                   model_name, train_generator, model)